# Workload聚合数据-多压力

In [ ]:
import sys
sys.path.append('./tools')

from aggregation import *
import analyze
import display

In [ ]:
exp_root = "/home/ict/appProfile/data/redis_1"
senario = "same_numa"
qos_columns = [
    "app_redis_qos_qps_of_redis_get",
    "app_redis_qos_qps_of_redis_total",
    "app_redis_qos_qps_of_redis_set",
    "app_redis_qos_p99_latency_set",
    "app_redis_qos_p99_latency_get",
]



corr_f = analyze.pearson_correlation
exp_roots = exp_roots_from_dir(exp_root, senario)

In [ ]:
no_stress_exp_root = exp_roots["no_stress"]
exp_root = exp_roots["stresses"][0]

no_stress_exp_data = read_from_dir(no_stress_exp_root)
no_stress_exp_data.set_workload_preprocess_funcs([
    filter_column_startswith(col_prefix=("vm", "app")),
    filter_column_useless(),
    filter_row_noise(col_prefix=("app")),
])

no_stress_df_epoch = no_stress_exp_data.agg_epoch()
workload_keys = list(no_stress_df_epoch.index)
workload_keys

In [ ]:
# 选择workload
workload_key = workload_keys[8]
# 选择QoS指标
qos_column = qos_columns[3]

# 一、干扰敏感度分析

## 1. Workload对于不同干扰的箱线图

In [ ]:
for exp_root in exp_roots["stresses"]:
    exp_data = read_from_dir(exp_root)
    exp_data.set_workload_preprocess_funcs([
        filter_column_startswith(col_prefix=("stress", "vm", "app")),
        filter_column_useless(excol_prefix=("stress")),
        filter_row_noise(col_prefix=("app")),
    ])


    df = exp_data.one_column_on_stresses(qos_column, workload_key)
    display.plt_box(df, "", qos_column)

## 2. 不同Workload下的劣化程度

In [ ]:
dfs = []
for exp_root in exp_roots["stresses"]:
    exp_data = read_from_dir(exp_root)
    df = agg_per_workload_column_on_stress(exp_data, no_stress_df_epoch, qos_column, stress="stress_cpu-load")
    dfs.append(df)
det_df = pd.concat(dfs)
det_df

In [ ]:
display.plt_deterioration_heatmap(det_df, qos_column)